In [3]:
# from transformers import BitsAndBytesConfig
# from transformers import LlamaForCausalLM, LlamaConfig, LlamaForSequenceClassification, AutoTokenizer
# from peft import LoraConfig, get_peft_model, get_peft_model_state_dict, \
#     set_peft_model_state_dict
# import torch
# import torch.nn as nn

# lora_target_modules = [
#     "q_proj",
#     "k_proj",
#     "v_proj",
#     "o_proj"
#     # "q_proj_lora",
#     # "k_proj_lora",
#     # "v_proj_lora",
# ]

# # 🎈=========================

# config = LoraConfig(r=8,
#                     lora_alpha= 32,
#                     target_modules=lora_target_modules,
#                     lora_dropout=0.2,
#                     bias="none",
#                     task_type="SEQ_CLS",
#                     inference_mode=False,
#                     )

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# # Load the model from the checkpoint
# checkpoint_path = "./checkpoint-2000"
# model = LlamaForSequenceClassification.from_pretrained(
#     checkpoint_path,
#     num_labels=2,
#     low_cpu_mem_usage=True,
#     quantization_config=bnb_config
# )

# score_weights = torch.load("./checkpoint-2000" + "/score.original_module.pt", map_location='cpu')
# model.score.original_module.load_state_dict(score_weights)

# tokenizer = AutoTokenizer.from_pretrained("./test")

# # Wrap with DataParallel if multiple GPUs are available
# if torch.cuda.device_count() > 1:
#     model = nn.DataParallel(model)

# # Apply LoRA configuration
# model = get_peft_model(model, config)
# model = model.half()  # Convert to half precision

# # Move model to GPU(s)
# model.cuda()

# # Ensure only LoRA parameters require gradients
# for name, param in model.named_parameters():
#     if "lora" in name:
#         param.requires_grad = True
#     else:
#         param.requires_grad = False

# # Move model to bfloat16 precision
# model = model.to(torch.bfloat16)

Loading checkpoint shards: 100%|██████████| 3/3 [00:10<00:00,  3.52s/it]
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at beomi/KoAlpaca-llama-1-7b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# 필요한 라이브러리 임포트
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, LlamaForSequenceClassification

# 모델과 토크나이저 불러오기
model = AutoModelForSequenceClassification.from_pretrained("./test")
# model = LlamaForSequenceClassification.from_pretrained(args["output_dir"]+"/test")
tokenizer = AutoTokenizer.from_pretrained("./test")


# Move model to bfloat16 precision
model = model.to(torch.bfloat16)
# Move model to GPU(s)
model.cuda()

Loading checkpoint shards:  33%|███▎      | 1/3 [00:03<00:06,  3.37s/it]

: 

In [2]:
# import gc

# # 모델을 CPU로 이동시키고 변수 삭제
# model.to('cpu')
# del model
# del tokenizer

# # 모든 변수 삭제
# for obj in gc.get_objects():
#     if torch.is_tensor(obj):
#         del obj

# gc.collect()
# torch.cuda.empty_cache()

/home/aiv-au/anaconda3/envs/minkyung_GLRec/lib/python3.11/site-packages/torch/distributed/distributed_c10d.py:366: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(


In [3]:
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [2]:
sentences = [
"""
### 질문: 7606번의 유저에게 이 A를 추천할까요? 아니면 B를 추천할까요?

### 맥락:7606번의 유저는 성별:남\/연령대:30대\/특이사항:없음\/관심 있는 건강고민 정보:피부건강,스트레스&수면,혈당,호흡기건강\/관심 있는 영양제 효능 정보:피로 개선의 특성을 가지고 있다.
7606번의 유저는'센트룸_(단종) 멀티구미 남성용'라는 이름의 영양제를 구매한 이력이 있다.
이 영양제의 효능:피로 개선,감기 덜 걸림,스트레스 감소\/이 영양제와 관련 있는 건강고민 정보:피부건강_피부손상보호
이 영양제를 구매한 또 다른 사람은 성별:남\/연령대:40대\/특이사항:없음\/관심 있는 건강고민 정보:갑상선건강,피로감,면역기능,치아&잇몸\/관심 있는 영양제 효능 정보:피로 개선의 특성을 지난다.
7606번의 유저는'해비추얼(와이즐리)_활력충전 멀티비타민&미네랄'라는 이름의 영양제를 구매한 이력이 있다.
이 영양제의 효능:피로 개선,감기 덜 걸림,스트레스 감소\/이 영양제와 관련 있는 건강고민 정보:갑상선건강,피부건강_피부손상보호,피부건강_피부재생,혈당_인슐린작용개선
이 영양제를 구매한 또 다른 사람은 성별:남\/연령대:40대\/특이사항:없음\/관심 있는 건강고민 정보:피부건강,갑상선건강,뼈건강,호흡기건강\/관심 있는 영양제 효능 정보:피로 개선의 특성을 지난다.
7606번의 유저가 구매를 고민하고 있는 영양제 A와 B의 정보는 다음과 같다:
'A'라는 이름의 영양제는 다음과 같은 특성을 가진다.
이 영양제의 효능:피로 개선,눈 떨림 개선,수면질 개선\/이 영양제와 관련 있는 건강고민 정보:뼈건강,임산부&태아건강,치아&잇몸_치아건강.
'B'라는 이름의 영양제는 다음과 같은 특성을 가진다.
이 영양제의 효능:피로 개선,감기 덜 걸림,수면질 개선\/이 영양제와 관련 있는 건강고민 정보:눈건강_안구건조개선,눈건강_야맹증개선,뼈건강,스트레스&수면_긴장완화,여성건강_생리전증후군&생리통개선,치아&잇몸_치아건강,치아잇몸&잇몸건강,피부건강_피부손상보호,혈당_인슐린작용개선.

### 답변:
""".strip(), #B
"""
### 질문: 25221번의 유저에게 이 A를 추천할까요? 아니면 B를 추천할까요?\n\n### 맥락:25221번의 유저는 성별:여\/연령대:30대\/특이사항:없음\/관심 있는 건강고민 정보:혈중중성지방,스트레스&수면,두뇌활동,눈건강,혈관혈액순환,임산부&태아건강\/관심 있는 영양제 효능 정보:혈압 수치 개선,눈 떨림 개선,학습 능력 향상,안구건조증 개선의 특성을 가지고 있다.\n25221번의 유저는'노르딕내추럴스_얼티메이트 오메가(소프트젤)'라는 이름의 영양제를 구매한 이력이 있다.\n이 영양제의 효능:피로 개선,콜레스테롤 수치 개선,얼굴색 좋아짐\/이 영양제와 관련 있는 건강고민 정보:눈건강_안구건조개선,두뇌활동,스트레스&수면_우울감개선,임산부&태아건강,혈관혈액순환_혈액순환개선,혈중중성지방_중성지방합성억제\n이 영양제를 구매한 또 다른 사람은 성별:남\/연령대:30대\/특이사항:없음\/관심 있는 건강고민 정보:혈중중성지방,스트레스&수면,두뇌활동,눈건강,혈관혈액순환\/관심 있는 영양제 효능 정보:어지러움 개선,콜레스테롤 수치 개선,두통 완화,안구건조증 개선의 특성을 지난다.\n25221번의 유저는'해비추얼(와이즐리)_(단종) 혈행 눈건강 기억력 오메가3'라는 이름의 영양제를 구매한 이력이 있다.\n이 영양제의 효능:피로 개선,집중력 개선,눈 피로감 개선\/이 영양제와 관련 있는 건강고민 정보:두뇌활동,스트레스&수면_우울감개선,혈중중성지방_중성지방합성억제\n이 영양제를 구매한 또 다른 사람은 성별:남\/연령대:40대\/특이사항:없음\/관심 있는 건강고민 정보:혈중중성지방,스트레스&수면,두뇌활동,눈건강,혈관혈액순환\/관심 있는 영양제 효능 정보:수면질 개선,혈당 수치 개선,눈 떨림 개선,콜레스테롤 수치 개선의 특성을 지난다.\n25221번의 유저가 구매를 고민하고 있는 영양제 A와 B의 정보는 다음과 같다:\n'A'라는 이름의 영양제는 다음과 같은 특성을 가진다.\n이 영양제의 효능:콜레스테롤 수치 개선,저림 개선,수면질 개선\/이 영양제와 관련 있는 건강고민 정보:혈관혈액순환_혈액순환개선,혈중중성지방_중성지방합성억제.\n'B'라는 이름의 영양제는 다음과 같은 특성을 가진다.\n이 영양제의 효능:감기 덜 걸림,비타민 수치 개선,구내염 개선\/이 영양제와 관련 있는 건강고민 정보:뼈건강_칼슘흡수촉진,호흡기건강.\n\n### 답변:
""".strip(), #A
"""
### 질문: 7606번의 유저에게 이 A를 추천할까요? 아니면 B를 추천할까요?\n\n### 맥락:7606번의 유저는 성별:남\/연령대:30대\/특이사항:없음\/관심 있는 건강고민 정보:피부건강,스트레스&수면,혈당,호흡기건강\/관심 있는 영양제 효능 정보:피로 개선의 특성을 가지고 있다.\n7606번의 유저는'해비추얼(와이즐리)_에너지 포커스'라는 이름의 영양제를 구매한 이력이 있다.\n이 영양제의 효능:피로 개선,감기 덜 걸림,구내염 개선\/이 영양제와 관련 있는 건강고민 정보:갑상선건강,피부건강_피부손상보호,피부건강_피부재생,혈당_인슐린작용개선\n이 영양제를 구매한 또 다른 사람은 성별:남\/연령대:40대\/특이사항:없음\/관심 있는 건강고민 정보:피부건강,뼈건강,눈건강,혈관혈액순환\/관심 있는 영양제 효능 정보:피로 개선의 특성을 지난다.\n7606번의 유저는'더리틀스_맥스데이 멀티부스터'라는 이름의 영양제를 구매한 이력이 있다.\n이 영양제의 효능:피로 개선,감기 덜 걸림,눈 피로감 개선\/이 영양제와 관련 있는 건강고민 정보:피로감,피부건강_피부재생,혈관혈액순환_호모시스테인균형\n이 영양제를 구매한 또 다른 사람은 성별:남\/연령대:30대\/특이사항:없음\/관심 있는 건강고민 정보:스트레스&수면,치아&잇몸,혈당,눈건강\/관심 있는 영양제 효능 정보:피로 개선의 특성을 지난다.\n7606번의 유저가 구매를 고민하고 있는 영양제 A와 B의 정보는 다음과 같다:\n'B'라는 이름의 영양제는 다음과 같은 특성을 가진다.\n이 영양제의 효능:피로 개선,감기 덜 걸림,구내염 개선\/이 영양제와 관련 있는 건강고민 정보:간건강,면역기능,빈혈_혈액생성,뼈건강_칼슘흡수촉진,소화&위식도건강_담즙분비촉진,피로감,피부건강_피부재생,혈관혈액순환_호모시스테인균형,호흡기건강.\n'A'라는 이름의 영양제는 다음과 같은 특성을 가진다.\n이 영양제의 효능:피로 개선,감기 덜 걸림,구내염 개선\/이 영양제와 관련 있는 건강고민 정보:갑상선건강,면역기능,뼈건강,치아&잇몸_치아건강,피부건강_피부손상보호,혈당_인슐린작용개선.\n\n### 답변:
""".strip(), #A
"""
### 질문: 10935번의 유저에게 이 A를 추천할까요? 아니면 B를 추천할까요?\n\n### 맥락:10935번의 유저는 성별:남\/연령대:40대\/특이사항:없음\/관심 있는 건강고민 정보:갑상선건강,치아잇몸&잇몸건강,스트레스&수면,뼈건강\/관심 있는 영양제 효능 정보:피로 개선의 특성을 가지고 있다.\n10935번의 유저는'슈퍼뉴트리션_슈퍼 이뮨'라는 이름의 영양제를 구매한 이력이 있다.\n이 영양제의 효능:피로 개선,감기 덜 걸림,스트레스 감소\/이 영양제와 관련 있는 건강고민 정보:면역기능,빈혈_혈액생성,뼈건강,뼈건강_칼슘흡수촉진,스트레스&수면_긴장완화,여성건강_생리전증후군&생리통개선,임산부&태아건강,치아&잇몸_치아건강,치아잇몸&잇몸건강,피로감,피부건강_피부손상보호,피부건강_피부재생,혈관혈액순환_호모시스테인균형,혈당_인슐린작용개선\n이 영양제를 구매한 또 다른 사람은 성별:여\/연령대:20대\/특이사항:없음\/관심 있는 건강고민 정보:빈혈,스트레스&수면,호흡기건강,임산부&태아건강\/관심 있는 영양제 효능 정보:눈 떨림 개선의 특성을 지난다.\n10935번의 유저는'닥터스베스트_멀티비타민'라는 이름의 영양제를 구매한 이력이 있다.\n이 영양제의 효능:피로 개선,감기 덜 걸림,수면질 개선\/이 영양제와 관련 있는 건강고민 정보:간건강,눈건강_야맹증개선,면역기능,빈혈_혈액생성,뼈건강,뼈건강_칼슘흡수촉진,피로감,피부건강_피부손상보호,피부건강_피부재생,혈관혈액순환_호모시스테인균형,혈당_인슐린작용개선,혈압_체액농도밸런스개선,호흡기건강\n이 영양제를 구매한 또 다른 사람은 성별:남\/연령대:20대\/특이사항:없음\/관심 있는 건강고민 정보:피로감,혈당,뼈건강,호흡기건강\/관심 있는 영양제 효능 정보:구내염 개선의 특성을 지난다.\n10935번의 유저가 구매를 고민하고 있는 영양제 A와 B의 정보는 다음과 같다:\n'B'라는 이름의 영양제는 다음과 같은 특성을 가진다.\n이 영양제의 효능:피로 개선,감기 덜 걸림,구내염 개선\/이 영양제와 관련 있는 건강고민 정보:갑상선건강,면역기능,빈혈_혈액생성,뼈건강,뼈건강_칼슘흡수촉진,치아&잇몸_치아건강,피부건강_피부재생,호흡기건강.\n'A'라는 이름의 영양제는 다음과 같은 특성을 가진다.\n이 영양제의 효능:피로 개선,감기 덜 걸림,구내염 개선\/이 영양제와 관련 있는 건강고민 정보:간건강,면역기능,빈혈_혈액생성,피로감,피부건강_피부재생,혈관혈액순환_호모시스테인균형.\n\n### 답변:
""".strip() #B

]

# A가 1, B가 0

In [5]:
import pandas as pd

item_info = pd.read_excel('item_info.xlsx')
# item_agg_exp = pd.read_excel('item_agg_exp_df.xlsx')
user_item = pd.read_excel('./user_item.xlsx')

In [ ]:
'안국건강_안국 루테인 지아잔틴 미니'
'암웨이 뉴트리라이트 밸런스 위드인 365 프로바이오틱스'

In [6]:
basic_prompt = """
### 질문: 7606번의 유저에게 이 A를 추천할까요? 아니면 B를 추천할까요?

### 맥락:7606번의 유저는 성별:여\/연령대:20대\/특이사항:없음\/관심 있는 건강고민 정보:눈건강,장건강,스트레스&수면\/관심 있는 영양제 효능 정보:스트레스&수면_긴장완화,눈건강_안구건조개선,장건강_배변활동의 특성을 가지고 있다.
7606번의 유저는'안국건강_안국 루테인 지아잔틴 미니'라는 이름의 영양제를 구매한 이력이 있다.
이 영양제의 효능:눈 피로감 개선,안구건조증 개선,야맹증 개선/이 영양제와 관련 있는 건강고민 정보:눈건강_황반색소유지
이 영양제를 구매한 또 다른 사람은 성별:남\/연령대:10대\/특이사항:없음\/관심 있는 건강고민 정보:눈건강\/관심 있는 영양제 효능 정보:눈 피로감 개선,시력 개선,야맹증 개선,눈 가려움 완화,안구건조증 개선의 특성을 지난다.
7606번의 유저는'암웨이 뉴트리라이트_밸런스 위드인 365 프로바이오틱스'라는 이름의 영양제를 구매한 이력이 있다.
이 영양제의 효능:변비 개선,복부 가스 덜 참,설사 빈도 감소/이 영양제와 관련 있는 건강고민 정보:장건강_유익균유해균균형도움
이 영양제를 구매한 또 다른 사람은 성별:남\/연령대:20대\/특이사항:없음\/관심 있는 건강고민 정보:피부건강,장건강\/관심 있는 영양제 효능 정보:아랫배 통증 완화,설사 빈도 감소,뾰루지 감소의 특성을 지난다.
7606번의 유저가 구매를 고민하고 있는 영양제 A와 B의 정보는 다음과 같다:
""".strip()

In [3]:
item_info

,브랜드명_제품명,제품효과,건강 고민 정보,item_info
0,21세기센트리_L-아르기닌 1000mg,"수면질 개선,피로 개선,근육통 완화","남성건강_정자운동성향상,운동능력&근육량_운동능력개선","이 영양제의 효능:수면질 개선,피로 개선,근육통 완화/이 영양제와 관련 있는 건강고..."
1,21세기센트리_마그네슘 250mg,"피로 개선,눈 떨림 개선,수면질 개선","스트레스&수면_긴장완화,여성건강_생리전증후군&생리통개선","이 영양제의 효능:피로 개선,눈 떨림 개선,수면질 개선/이 영양제와 관련 있는 건강..."
2,21세기센트리_멀티비타민 & 멀티미네랄 어덜트,"피로 개선,감기 덜 걸림,구내염 개선","갑상선건강,눈건강_야맹증개선,면역기능,뼈건강_칼슘흡수촉진,피부건강_피부손상보호,혈당...","이 영양제의 효능:피로 개선,감기 덜 걸림,구내염 개선/이 영양제와 관련 있는 건강..."
3,21세기센트리_칼슘 마그네슘 징크 + D3,"피로 개선,눈 떨림 개선,수면질 개선","스트레스&수면_긴장완화,여성건강_생리전증후군&생리통개선,치아&잇몸_치아건강","이 영양제의 효능:피로 개선,눈 떨림 개선,수면질 개선/이 영양제와 관련 있는 건강..."
4,CJ제일제당_BYO 바이오 20억 생유산균,"변비 개선,복부 가스 덜 참,설사 빈도 감소","임산부&태아건강,장건강_유익균유해균균형도움","이 영양제의 효능:변비 개선,복부 가스 덜 참,설사 빈도 감소/이 영양제와 관련 있..."
...,...,...,...,...
729,홀리데이즈_L-아르기닌 1000,"운동 능력 증가,운동 후 회복력 좋아짐,수면질 개선","남성건강_정자운동성향상,운동능력&근육량_운동능력개선","이 영양제의 효능:운동 능력 증가,운동 후 회복력 좋아짐,수면질 개선/이 영양제와 ..."
730,홀리데이즈_오메가3 1100,"피로 개선,콜레스테롤 수치 개선,수면질 개선",스트레스&수면_우울감개선,"이 영양제의 효능:피로 개선,콜레스테롤 수치 개선,수면질 개선/이 영양제와 관련 있..."
731,황금구렁이_황금구렁이,"변비 개선,복부 가스 덜 참,설사 빈도 감소",장건강_배변활동,"이 영양제의 효능:변비 개선,복부 가스 덜 참,설사 빈도 감소/이 영양제와 관련 있..."
732,휴럼_시서스 스피드 다이어트,"식욕 감소,근육량 증가,체중 감소",체지방_지방대사촉진,"이 영양제의 효능:식욕 감소,근육량 증가,체중 감소/이 영양제와 관련 있는 건강고민..."


In [18]:
item_info_value = item_info.loc[item_info['브랜드명_제품명'] == '휴온스_메리트 C&D', 'item_info'].values[0]

In [14]:
compare = """
'B'라는 이름의 영양제는 다음과 같은 특성을 가진다.
이 영양제의 효능:피로 개선,감기 덜 걸림,수면질 개선\/이 영양제와 관련 있는 건강고민 정보:눈건강_안구건조개선,눈건강_야맹증개선,뼈건강,스트레스&수면_긴장완화,여성건강_생리전증후군&생리통개선,치아&잇몸_치아건강,치아잇몸&잇몸건강,피부건강_피부손상보호,혈당_인슐린작용개선.
'A'라는 이름의 영양제는 다음과 같은 특성을 가진다.
이 영양제의 효능:피로 개선,눈 떨림 개선,수면질 개선\/이 영양제와 관련 있는 건강고민 정보:뼈건강,임산부&태아건강,치아&잇몸_치아건강.
### 답변:
"""

In [15]:
sentences = [basic_prompt + compare]

In [20]:
import random
import numpy as np

# Set the seed for generating random numbers
seed = 42

# Set the seed for Python's built-in random module
random.seed(seed)

# Set the seed for numpy
np.random.seed(seed)

# Set the seed for PyTorch (both CPU and CUDA)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# Set the CuDNN backend to deterministic
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

for sentence in sentences:
    # 배치 인퍼런스를 위한 입력 텍스트
    input_texts = sentence

    # 토크나이저로 입력 텍스트들을 배치로 변환
    inputs = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True)

    # GPU로 이동
    inputs = {key: val.cuda() for key, val in inputs.items()}

    # 모델 예측
    with torch.inference_mode():
        outputs = model(**inputs, output_attentions=True)

    # 결과값(로짓)을 소프트맥스 함수에 넣어서 확률로 변환
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)
    print(probabilities)

    # 가장 높은 확률을 가진 라벨 선택
    predicted_labels = torch.argmax(probabilities, dim=-1).cpu().numpy()

    print(f"Predicted labels: {predicted_labels}")

    # 텐서 해제 및 GPU 메모리 정리
    del inputs
    del outputs
    del logits
    del probabilities
    torch.cuda.empty_cache()

tensor([[0.7188, 0.2812]], device='cuda:0', dtype=torch.bfloat16)
Predicted labels: [0]


In [21]:
predicted_labels[0]

0